In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interactive_output

In [2]:
%matplotlib inline

In [3]:
df = pd.read_csv('turma_matricula_docente_filtrados.csv')

In [4]:
# Widgets
unidade_dropdown = widgets.Dropdown(
    options=df[['unidade_responsavel', 'unidade_responsavel']].drop_duplicates().values.tolist(),
    description='Curso:',
    value=None
)

componente_dropdown = widgets.Dropdown(
    description='Componente Curricular:',
    value=None
)

docente_dropdown = widgets.Dropdown(
    description='Docente:',
    value=None
)

In [5]:
def plot_pie_chart(data):
    contagem_descricao = data['descricao'].value_counts()
    porcentagens = (contagem_descricao / contagem_descricao.sum()) * 100
    
    fig, ax = plt.subplots(figsize=(10, 6))
    wedges, texts, autotexts = ax.pie(porcentagens, labels=porcentagens.index, autopct='%1.1f%%', startangle=90, wedgeprops=dict(width=0.4))
    ax.axis('equal')
    
    for text, autotext in zip(texts, autotexts):
        text.set(size=10)
        autotext.set(size=10)
    
    legend_labels = [f"{label}: {value}" for label, value in zip(contagem_descricao.index, contagem_descricao.values)]
    ax.legend(wedges, legend_labels, title="Descrições", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    
    plt.show()

In [6]:
def update_componentes(*args):
    df_unidade = df[df['unidade_responsavel'] == unidade_dropdown.value]
    componente_dropdown.options = pd.unique(df_unidade['nome_componente']).tolist()
    componente_dropdown.value = componente_dropdown.options[0] if componente_dropdown.options else None

In [7]:
def update_docentes(*args):
    df_unidade = df[df['unidade_responsavel'] == unidade_dropdown.value]
    df_componente = df_unidade[df_unidade['nome_componente'] == componente_dropdown.value]
    docente_dropdown.options = pd.unique(df_componente['nome_docente']).tolist()
    docente_dropdown.value = docente_dropdown.options[0] if docente_dropdown.options else None

In [8]:
def plot_graph_based_on_docente(*args):
    with output:
        clear_output(wait=True)
        
        df_unidade = df[df['unidade_responsavel'] == unidade_dropdown.value]
        df_componente = df_unidade[df_unidade['nome_componente'] == componente_dropdown.value]
        df_docente = df_componente[df_componente['nome_docente'] == docente_dropdown.value]
        
        df_filtrado = df_docente.drop_duplicates(subset='discente')
        plot_pie_chart(df_filtrado)

In [9]:
unidade_dropdown = widgets.Dropdown(
    options=pd.unique(df['unidade_responsavel']).tolist(),
    value=pd.unique(df['unidade_responsavel']).tolist()[0],
    description='Curso:',
)

componente_dropdown = widgets.Dropdown(description='Componente:')
docente_dropdown = widgets.Dropdown(description='Docente:')

unidade_dropdown.observe(update_componentes, 'value')
componente_dropdown.observe(update_docentes, 'value')
docente_dropdown.observe(plot_graph_based_on_docente, 'value')

In [10]:
input_widgets = widgets.VBox([unidade_dropdown, componente_dropdown, docente_dropdown])
output = widgets.Output()
app = widgets.VBox([input_widgets, output])

update_componentes()
update_docentes()
plot_graph_based_on_docente()
display(app)